# Get cell processing info

In [ ]:
import os
import sys
from pathlib import Path
import pandas as pd
import geopandas as gpd
import numpy as np
import pyproj
import pickle
from shapely.geometry import box
from IPython.display import Image
from ipywidgets import Label
import math
import matplotlib
#import jupyter_compare_view

%matplotlib inline
#matplotlib.use('nbagg')
import matplotlib.pyplot as plt

In [ ]:
sys.path.append(r"../LUCinSA_helpers")
from file_checks import *
from plot import *

In [ ]:
'''
PARAMETERS: modify in notebook_params notebook, then run that notebook and this cell to update here
DO NOT modify this cell
'''

%store -r basic_config
print("Basic Parameters: \n brdf_dir = {} \n gridCell = {} \n index_dir = {} \n home_dir = {} \n yr_range = {} \n image_type = {}"
      .format(basic_config['brdf_dir'],basic_config['grid_cell'],basic_config['index_dir'],basic_config['home_dir'],basic_config['yr_range'],basic_config['image_type']))

## Read processing.info file and get coreg info

In [ ]:
db_path = Path('{}/{:06d}/processing.info'.format(basic_config['raw_dir'],int(basic_config['grid_cell'])))
p_df = read_db(db_path,'current')
p_df = p_df.reset_index()
p_df['shift'] = p_df.apply(lambda x: math.sqrt(math.pow(x['shift_x'],2)+math.pow(x['shift_y'],2)),axis=1)
p_df.index = pd.to_datetime(p_df['date'], format='%Y%m%d', errors='coerce')
p_df.head(n=10)

In [ ]:
p_df5 = p_df[p_df['sensor']=='lt05']
creg_l5 = p_df5[p_df5['coreg']==True]
print('of the {} L5 images, {} were coregistered'.format(p_df5.shape[0],creg_l5.shape[0]))
avg_shift = creg_l5['shift'].mean()
med_shift = creg_l5['shift'].median()
max_shift = creg_l5['shift'].max()
print('Average shift: {}. Median shift: {}. Max shift: {}'.format(avg_shift,med_shift,max_shift))
p_df5.head(n=5)

In [ ]:
p_df7 = p_df[p_df['sensor']=='le07']
creg_l7 = p_df7[p_df7['coreg']==True]
print('of the {} L7 images, {} were coregistered'.format(p_df7.shape[0],creg_l7.shape[0]))
avg_shift = creg_l7['shift'].mean()
med_shift = creg_l7['shift'].median()
max_shift = creg_l7['shift'].max()
print('Average shift: {}. Median shift: {}. Max shift: {}'.format(avg_shift,med_shift,max_shift))
p_df7.head(n=5)

In [ ]:
p_dfs = p_df[p_df['sensor'].str.startswith('s')]
creg_s = p_dfs[p_dfs['coreg']==True]
print('of the {} Sentinel images, {} were coregistered'.format(p_dfs.shape[0],creg_s.shape[0]))
avg_shift = creg_s['shift'].mean()
med_shift = creg_s['shift'].median()
max_shift = creg_s['shift'].max()
print('Average shift: {}. Median shift: {}. Max shift: {}'.format(avg_shift,med_shift,max_shift))
p_dfs.head(n=5)

## Plot coreg time series

In [ ]:
p_df.groupby('sensor')['shift'].plot(legend='True')
plt.ylim(0,2)
plt.ylim([.2,.5])
plt.show()


## View uncoreged images against Landsat reference

In [ ]:
def open_img(img):
    if img.endswith('.tif'):
        with rio.open(img) as src:
             img_out = src.read(1)
    elif img.endswith('.nc'):
        with xr.open_dataset(img) as xrimg:
            xrcrs = xrimg.crs
        band = xrimg['nir'].where(xrimg['nir'] != 65535)
        #xr_idx = xrimg[band]
        #img_out = xr_idx.where(xr_idx > 0)
        img_out = band

    return img_out

def crop(img, x1, x2, y1, y2):

    mask = np.zeros(img.shape)
    mask[y1:y2+1, x1:x2+1]=1
    m = mask>0

    return img[m].reshape((y2+1-y1, x2+1-x1))

def handle_interaction(**kwargs):
    if kwargs.get('type') == 'click':
        label.value = str(kwargs.get('coordinates'))
        coords =eval(label.value) 
        coordList.append(coords)
        print(coordList)


In [ ]:
ref_img = '{}/{:06d}/brdf_ref/_tmp_reference.tif'.format(basic_config['raw_dir'],int(basic_config['grid_cell']))
ref = open_img(ref_img)

p_df5 = p_df5.sort_values(by=['shift'], ascending=False)
imgs_l5 = p_df5['brdf_id'].tolist()
#print(imgs_l5)
comp_img = '{}/{:06d}/brdf/{}_coreg.nc'.format(basic_config['raw_dir'],int(basic_config['grid_cell']),imgs_l5[1].strip('.nc'))
print(comp_img)
comp = open_img(comp_img)

## Need to run pip install jupyter_compare_view
from jupyter_compare_view import compare
compare(ref, comp, cmap="gray", start_mode="horizontal", start_slider_pos=0.73)
plt.figure(figsize =(10,10))
plt.imshow(ref)
plt.set_title("ref_image")
plt.axis('off')
plt.imshow(comp)
plt.set_title("coregistered")
plt.axis('off');

## To save an html copy of this notebook with all outputs:

In [ ]:
### Run to print output as html
out_name = str(basic_config['country']+'1p_ExploreProcessing_coregistration_'+str(basic_config['grid_cell']))
!jupyter nbconvert --output-dir='./Outputs' --to html --no-input --output=$out_name 1p_ExploreProcessing_coregistration.ipynb